In [6]:
!pip install scikit-learn
!pip install tqdm
!pip install oddt
!pip install h5py

  Using cached numpydoc-1.8.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached sphinx-8.0.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached sphinxcontrib_applehelp-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached sphinxcontrib_devhelp-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached sphinxcontrib_jsmath-1.0.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached sphinxcontrib_htmlhelp-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached sphinxcontrib_serializinghtml-2.0.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached sphinxcontrib_qthelp-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached pygments-2.18.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached docutils-0.21.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached snowballstemmer-2.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached babel-2.16.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached alabaster-1.0.0-py3-none-any.whl.metadata

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdb2pqr 3.6.1 requires docutils<0.18, but you have docutils 0.21.2 which is incompatible.


In [18]:
!pip install tensorflow
!pip install torch

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     --------------------------------- ------ 41.0/48.7 kB 2.0 MB/s eta 0:00:01
     --------------------------

In [19]:
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences

In [8]:
import htmd.ui as ht
from moleculekit.molecule import Molecule
import moleculekit.tools.voxeldescriptors as vd
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors, viewVoxelFeatures
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping
from moleculekit.smallmol.smallmol import SmallMol
from moleculekit.home import home
import os

import csv
from tqdm import *
import os
import pickle
import numpy as np
import pandas as ps
import multiprocessing as mp
from sklearn.model_selection import train_test_split
from oddt import toolkit
from oddt import datasets
from oddt.datasets import pdbbind

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [9]:
data_dir = ".\DLSCORE-CNN-master\DLSCORE-CNN-master\dataset"
pdbbind_dir = os.path.join(data_dir, "refined-set-2016\\")
pdbbind_dataset = pdbbind(home = pdbbind_dir, default_set='refined', version=2016)

### Manual Functions for feature extractions

In [11]:
# (c) 2015-2018 Acellera Ltd http://www.acellera.com
# All Rights Reserved
# Distributed under HTMD Software License Agreement
# No redistribution in whole or part
#


"""
All values below are taken from PeriodicTable.C from VMD source code

van der Waals radii are taken from A. Bondi,
J. Phys. Chem., 68, 441 - 452, 1964,
except the value for H, which is taken from R.S. Rowland & R. Taylor,
J.Phys.Chem., 100, 7384 - 7391, 1996. Radii that are not available in
either of these publications have RvdW = 2.00 �.
The radii for Ions (Na, K, Cl, Ca, Mg, and Cs are based on the CHARMM27
Rmin/2 parameters for (SOD, POT, CLA, CAL, MG, CES) by default.
"""

massdict = {'Ac': 227.0, 'Ag': 107.8682, 'Al': 26.981538, 'Am': 243.0, 'Ar': 39.948, 'As': 74.9216, 'At': 210.0,
            'Au': 196.96655, 'B': 10.811, 'Ba': 137.327, 'Be': 9.012182, 'Bh': 264.0, 'Bi': 208.98038, 'Bk': 247.0,
            'Br': 79.904, 'C': 12.0107, 'Ca': 40.078, 'Cd': 112.411, 'Ce': 140.116, 'Cf': 251.0, 'Cl': 35.453,
            'Cm': 247.0, 'Co': 58.9332, 'Cr': 51.9961, 'Cs': 132.90545, 'Cu': 63.546, 'Db': 262.0, 'Ds': 271.0,
            'Dy': 162.5, 'Er': 167.259, 'Es': 252.0, 'Eu': 151.964, 'F': 18.9984032, 'Fe': 55.845, 'Fm': 257.0,
            'Fr': 223.0, 'Ga': 69.723, 'Gd': 157.25, 'Ge': 72.64, 'H': 1.00794, 'He': 4.0026, 'Hf': 178.49,
            'Hg': 200.59, 'Ho': 164.93032, 'Hs': 269.0, 'I': 126.90447, 'In': 114.818, 'Ir': 192.217, 'K': 39.0983,
            'Kr': 83.798, 'La': 138.9055, 'Li': 6.941, 'Lr': 262.0, 'Lu': 174.967, 'Md': 258.0, 'Mg': 24.305,
            'Mn': 54.938049, 'Mo': 95.94, 'Mt': 268.0, 'N': 14.0067, 'Na': 22.98977, 'Nb': 92.90638, 'Nd': 144.24,
            'Ne': 20.1797, 'Ni': 58.6934, 'No': 259.0, 'Np': 237.0, 'O': 15.9994, 'Os': 190.23, 'P': 30.973761,
            'Pa': 231.03588, 'Pb': 207.2, 'Pd': 106.42, 'Pm': 145.0, 'Po': 209.0, 'Pr': 140.90765, 'Pt': 195.078,
            'Pu': 244.0, 'Ra': 226.0, 'Rb': 85.4678, 'Re': 186.207, 'Rf': 261.0, 'Rg': 272.0, 'Rh': 102.9055,
            'Rn': 222.0, 'Ru': 101.07, 'S': 32.065, 'Sb': 121.76, 'Sc': 44.95591, 'Se': 78.96, 'Sg': 266.0,
            'Si': 28.0855, 'Sm': 150.36, 'Sn': 118.71, 'Sr': 87.62, 'Ta': 180.9479, 'Tb': 158.92534, 'Tc': 98.0,
            'Te': 127.6, 'Th': 232.0381, 'Ti': 47.867, 'Tl': 204.3833, 'Tm': 168.93421, 'U': 238.02891, 'V': 50.9415,
            'W': 183.84, 'X': 0.0, 'Xe': 131.293, 'Y': 88.90585, 'Yb': 173.04, 'Zn': 65.409, 'Zr': 91.224}

radiidict = {'Ac': 2.0, 'Ag': 1.72, 'Al': 2.0, 'Am': 2.0, 'Ar': 1.88, 'As': 1.85, 'At': 2.0, 'Au': 1.66, 'B': 2.0,
             'Ba': 2.0, 'Be': 2.0, 'Bh': 2.0, 'Bi': 2.0, 'Bk': 2.0, 'Br': 1.85, 'C': 1.7, 'Ca': 1.37, 'Cd': 1.58,
             'Ce': 2.0, 'Cf': 2.0, 'Cl': 2.27, 'Cm': 2.0, 'Co': 2.0, 'Cr': 2.0, 'Cs': 2.1, 'Cu': 1.4, 'Db': 2.0,
             'Ds': 2.0, 'Dy': 2.0, 'Er': 2.0, 'Es': 2.0, 'Eu': 2.0, 'F': 1.47, 'Fe': 2.0, 'Fm': 2.0, 'Fr': 2.0,
             'Ga': 1.07, 'Gd': 2.0, 'Ge': 2.0, 'H': 1.2, 'He': 1.4, 'Hf': 2.0, 'Hg': 1.55, 'Ho': 2.0, 'Hs': 2.0,
             'I': 1.98, 'In': 1.93, 'Ir': 2.0, 'K': 1.76, 'Kr': 2.02, 'La': 2.0, 'Li': 1.82, 'Lr': 2.0, 'Lu': 2.0,
             'Md': 2.0, 'Mg': 1.18, 'Mn': 2.0, 'Mo': 2.0, 'Mt': 2.0, 'N': 1.55, 'Na': 1.36, 'Nb': 2.0, 'Nd': 2.0,
             'Ne': 1.54, 'Ni': 1.63, 'No': 2.0, 'Np': 2.0, 'O': 1.52, 'Os': 2.0, 'P': 1.8, 'Pa': 2.0, 'Pb': 2.02,
             'Pd': 1.63, 'Pm': 2.0, 'Po': 2.0, 'Pr': 2.0, 'Pt': 1.72, 'Pu': 2.0, 'Ra': 2.0, 'Rb': 2.0, 'Re': 2.0,
             'Rf': 2.0, 'Rg': 2.0, 'Rh': 2.0, 'Rn': 2.0, 'Ru': 2.0, 'S': 1.8, 'Sb': 2.0, 'Sc': 2.0, 'Se': 1.9,
             'Sg': 2.0, 'Si': 2.1, 'Sm': 2.0, 'Sn': 2.17, 'Sr': 2.0, 'Ta': 2.0, 'Tb': 2.0, 'Tc': 2.0, 'Te': 2.06,
             'Th': 2.0, 'Ti': 2.0, 'Tl': 1.96, 'Tm': 2.0, 'U': 1.86, 'V': 2.0, 'W': 2.0, 'X': 1.5, 'Xe': 2.16, 'Y': 2.0,
             'Yb': 2.0, 'Zn': 1.39, 'Zr': 2.0}


def massByElement(element):
    try:
        return massdict[element]
    except:
        return 0.


def radiusByElement(element):
    try:
        return radiidict[element]
    except:
        return 1.5


def _getRadii(mol):
    """ Gets vdW radius for each elem in mol.element. Source VMD.

    Parameters
    ----------
    mol :
        A Molecule object. Needs to be read from Autodock 4 .pdbqt format

    Returns
    -------
    radii : np.ndarray
        vdW radius for each element in mol.
    """

    mappings = {  # Mapping pdbqt representation to element.
        'HD': 'H',
        'HS': 'H',
        'A': 'C',
        'NA': 'N',
        'NS': 'N',
        'OA': 'O',
        'OS': 'O',
        'MG': 'Mg',
        'SA': 'S',
        'CL': 'Cl',
        'CA': 'Ca',
        'MN': 'Mn',
        'FE': 'Fe',
        'ZN': 'Zn',
        'BR': 'Br',
        'CS': 'C', # Extra added
        'PB': 'P',
        'Oa':'O'
        # 'Pb':'P',
        # 'Cs': 'C'
    }
    atoms = ['H', 'C', 'N', 'O', 'F', 'Mg', 'P', 'S', 'Cl', 'Ca', 'Fe', 'Zn', 'Br', 'I', 'Pb', 'Cs']
    atoms.extend(['Co', 'Hg', 'Ni', 'Sr', 'Mn', 'K', 'Se', 'Cu', 'Cd', 'Li', 'Na']) # Extra added
    for el in atoms:
        #print(f'mappings[el]: {mappings[el]}')
        mappings[el] = el
        #print(el)

    res = np.zeros(mol.numAtoms)
    for a in range(mol.numAtoms):
        elem = mol.element[a]
        #print(f'elem = {elem}')

        if elem not in mappings:
            raise ValueError('PDBQT element {} does not exist in mappings.'.format(elem))
        elem = mappings[elem]
        if elem in radiidict:
            rad = radiusByElement(elem)
        else:
            print('Unknown element -', mol.element[a], '- at atom index ', a)
            rad = 1.5
        res[a] = rad
    return res

   


In [12]:
def _getGridCenters(llc, N, resolution):
    xrange = [llc[0] + resolution * x for x in range(0, N[0])]
    yrange = [llc[1] + resolution * x for x in range(0, N[1])]
    zrange = [llc[2] + resolution * x for x in range(0, N[2])]
    centers = np.zeros((N[0], N[1], N[2], 3))
    for i, x in enumerate(xrange):
        for j, y in enumerate(yrange):
            for k, z in enumerate(zrange):
                centers[i, j, k, :] = np.array([x, y, z])
    return centers

### Hessain Matrix Generation

In [13]:
def read_pdb_file(pdb_file):
    coordinates = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM'):
                atom_type = line[12:16].strip()
                atom_no = line[23:27].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                b_factor = float(line[60:66])
                if (atom_type == 'CA'):
                    coordinates.append([atom_type, atom_no, x, y, z, b_factor])
    return coordinates
def build_Hessain(coordinates):
    no_of_residue = len(coordinates)
    cut_off = 18
    H = np.zeros((3*no_of_residue, 3*no_of_residue), float)
    for i in range(0, no_of_residue-1):
        cor_i = np.array(coordinates[i][2:5])
        res_i_x = i * 3
        res_i_z = res_i_x + 3
        cor_i_j = []
        p = i + 1
        for k in range(p, no_of_residue):
            temp = np.array(coordinates[k][2:5]) - cor_i
            cor_i_j.append(temp)
        cor_i_j = np.array(cor_i_j)
        for j, s_ij_2 in enumerate((np.power(cor_i_j, 2)).sum(1)):
            if s_ij_2 > (cut_off*cut_off):
                continue
            # Super element 3x3
            gamma = 1
            d_i_j = cor_i_j[j]
            j += i+1
            res_j_x = j * 3
            res_j_z = res_j_x + 3
            super_element = np.outer(d_i_j, d_i_j) * (-gamma / s_ij_2)
            H[res_i_x:res_i_z, res_j_x:res_j_z] = super_element
            H[res_j_x:res_j_z, res_i_x:res_i_z] = super_element
            H[res_i_x:res_i_z, res_i_x:res_i_z] = H[res_i_x:res_i_z, res_i_x:res_i_z] - super_element
            H[res_j_x:res_j_z, res_j_x:res_j_z] = H[res_j_x:res_j_z, res_j_x:res_j_z] - super_element

    return H


### Feature Extraction

In [14]:
def get_pdb_complex_feature(protein_file, ligand_file, pocket_file):
    """ Returns voxel features for a pdb complex """

    def get_prop(mol, left_most_point):
        """ Returns atom occupancies """
        n = [24, 24, 24] # Voxel size (size of big voxel or cube)
        
        # Get the channels
        channels = vd._getAtomtypePropertiesPDBQT(mol)
        #print(f'Cnalles = {channels}')
        sigmas = _getRadii(mol)
        #print(f'signmas = {sigmas}')
        channels = sigmas[:, np.newaxis] * channels.astype(float)
        
        # Choose the grid centers
        centers = _getGridCenters(llc = left_most_point, N = n, resolution=1)
        #print(f'centers = {centers}')
        centers = centers.reshape(np.prod(n), 3)
        
        # Extract the features and return
        features = vd._getOccupancyC(mol.coords[:, :, mol.frame], centers, channels)
        return features.reshape(*n, -1)

    def get_pocket_feature(pocket_file):
        corr = read_pdb_file(pocket_file)
        no_of_residue = len(corr)
        
        H = build_Hessain(corr)
    
        eigvals, eigvec = np.linalg.eigh(H)

        max_effect_mode = eigvec[:, -1] * eigvals[-1]

        #print(max_effect_mode.shape)

        return max_effect_mode #Basiaclly it is just a 1-D numpy array
    
    # Generate the HTMD Molecule objects
    protein_mol = Molecule(protein_file)
    ligand_mol = Molecule(ligand_file)

    left_most_point = list(np.mean(ligand_mol.coords.reshape(-1, 3), axis=0) - 12.0)
    
    # Get the features for both the protein and the ligand. Return those after concatenation.
    protein_features = get_prop(protein_mol, left_most_point)
    ligand_features = get_prop(ligand_mol, left_most_point)
    pocket_feature = get_pocket_feature(pocket_file)
    #print(pocket_feature.shape)

    protein_features_t = protein_features.transpose().reshape([8, 24, 24, 24])    
    ligand_features_t = protein_features.transpose().reshape([8, 24, 24, 24])    
    
    return np.concatenate((protein_features_t, ligand_features_t), axis=0), pocket_feature


In [15]:
def get_pdb_features(ids, sets="refined"):
    """ Returns features for given pdb ids"""
    pdb_ids = []
    pdb_features = []
    pocket_features = []

    for pdbid in tqdm(ids):
        protein_file = os.path.join(pdbbind_dir, pdbid, pdbid + "_protein.pdbqt")
        ligand_file = os.path.join(pdbbind_dir, pdbid, pdbid + "_ligand.pdbqt")
        pocket_file = os.path.join(pdbbind_dir, pdbid, pdbid + "_pocket.pdb")
        if not os.path.isfile(protein_file) or not os.path.isfile(ligand_file): 
            continue
            #print(1)
            
        try:
            features, pocket_feat = get_pdb_complex_feature(protein_file, ligand_file, pocket_file)
            #print(list(pocket_feat))
        except Exception as e:
            print("ERROR in ", pdbid , " ", str(e))
            #print(protein_file)
            continue

        #print(pdbid)
        #print(features) 
        
        pdb_ids.append(pdbid)
        pdb_features.append(features)
        pocket_features.append(list(pocket_feat))
        #print(len(pocket_features))
    
    # Convert the list of features as numpy array and return


    
    # print(len(pocket_features))
    # print(len(max(pocket_features,key = lambda x: len(x))))
    # data_x_2 = np.zeros([len(pocket_features),len(max(pocket_features,key = lambda x: len(x)))])
    # for i,j in enumerate(pocket_features):
    #     data_x_2[i][0:len(j)] = j


    pocket_features = pad_sequences(pocket_features, maxlen=256, padding='post', truncating ='post', value = 1)
    data_x_2 = np.array(pocket_features, dtype=np.float32)
    print(data_x_2.shape)
    
    data_x_1 = np.array(pdb_features, dtype=np.float32)
    data_y = np.array([pdbbind_dataset.sets[sets][_id] for _id in pdb_ids], dtype=np.float32)

    print(data_x_1.shape)
    

    return data_x_1, data_y, data_x_2


In [16]:
def get_features():
    """ Returns features for all the complexes in the dataset. """ 
    # List ids in the core set
    core_ids = list(pdbbind_dataset.sets['core'].keys())
    # List ids in the refined set
    refined_ids = list(pdbbind_dataset.sets['refined'].keys()) 
    # remove core ids from the refined set.
    refined_ids = [i for i in refined_ids if i not in core_ids]
    
    # Get the features 
    print("Extracting features for the core set")
    core_x_1, core_y, core_x_2 = get_pdb_features(core_ids, sets="core")
    print("Extracting features for the refined set")
    refined_x_1, refined_y, refined_x_2 = get_pdb_features(refined_ids)    
    
    return core_x_1, core_y, refined_x_1, refined_y, core_x_2, refined_x_2

In [20]:
import warnings
warnings.filterwarnings('ignore')
test_x_1, test_y, train_x_1, train_y, test_x_2, train_x_2 = get_features()

Extracting features for the core set


100%|████████████████████████████████████████████████████████████████████████████████| 290/290 [01:32<00:00,  3.14it/s]


(290, 256)
(290, 16, 24, 24, 24)
Extracting features for the refined set


 20%|███████████████▍                                                               | 739/3767 [04:23<14:45,  3.42it/s]

ERROR in  1mrn   Element A was read in file .\DLSCORE-CNN-master\DLSCORE-CNN-master\dataset\refined-set-2016\1mrn\1mrn_ligand.pdbqt but was not found in the periodictable. To disable this check, pass `validateElements=False` to the Molecule constructor or read method.


...

ERROR in  4az6   Element Oa was read in file .\DLSCORE-CNN-master\DLSCORE-CNN-master\dataset\refined-set-2016\4az6\4az6_ligand.pdbqt but was not found in the periodictable. To disable this check, pass `validateElements=False` to the Molecule constructor or read method.


100%|█████████████████████████████████████████████████████████████████████████████▋| 3752/3767 [20:48<00:03,  4.94it/s]

ERROR in  6cpa   Element A was read in file .\DLSCORE-CNN-master\DLSCORE-CNN-master\dataset\refined-set-2016\6cpa\6cpa_ligand.pdbqt but was not found in the periodictable. To disable this check, pass `validateElements=False` to the Molecule constructor or read method.


100%|██████████████████████████████████████████████████████████████████████████████| 3767/3767 [20:52<00:00,  3.01it/s]


(3747, 256)
(3747, 16, 24, 24, 24)


In [21]:
print(train_y.shape)
print(test_x_1.shape)
print(test_x_2.shape)
print(test_y.shape)
print(train_x_1.shape) # To pad length = 320
print(train_x_2.shape)

(3747,)
(290, 16, 24, 24, 24)
(290, 256)
(290,)
(3747, 16, 24, 24, 24)
(3747, 256)


### Splitting Data

In [22]:
train_x_1, valid_x_1, train_x_2, valid_x_2, train_y, valid_y = train_test_split(train_x_1, train_x_2, train_y, test_size=0.2)
print("Shapes in the training, test and the validation set: ", train_x_1.shape, train_x_2.shape, valid_x_1.shape, valid_x_2.shape, test_x_1.shape, test_x_2.shape)
# x_train, x_test, y_train, y_test = train_test_split(dataset_df['Sequences'], dataset_df['class'], test_size = 0.2)

Shapes in the training, test and the validation set:  (2997, 16, 24, 24, 24) (2997, 256) (750, 16, 24, 24, 24) (750, 256) (290, 16, 24, 24, 24) (290, 256)


### Saving Extracted Features

In [23]:
import h5py
print("Saving the data in data_dell.h5")
h5f = h5py.File(os.path.join(data_dir, "data_dell.h5"), 'w')
h5f.create_dataset('train_x_1', data=train_x_1)
h5f.create_dataset('train_x_2', data=train_x_2)
h5f.create_dataset('train_y', data=train_y)
h5f.create_dataset('valid_x_1', data=valid_x_1)
h5f.create_dataset('valid_x_2', data=valid_x_2)
h5f.create_dataset('valid_y', data=valid_y)
h5f.create_dataset('test_x_1', data=test_x_1)
h5f.create_dataset('test_x_2', data=test_x_2)
h5f.create_dataset('test_y', data=test_y)
h5f.close()

Saving the data in data_dell.h5
